# Phần 1: Thư viện

- **re**: Xử lý chuỗi với các biểu thức chính quy
- **demoji**: Biểu tượng cảm xúc
- **pickle**: Lưu trữ và truy xuất dữ liệu
- **numpy**: Tính toán 
- **pandas**: Cấu trúc dữ liệu và công cụ phân tích dữ liệu 
- **matplotlib.pyplot**: Biểu đồ trong Python
- **tensorflow**: Xây dựng và huấn luyện mô hình học máy
- **scikit-learn**: Các công cụ cho việc học máy và thống kê
- **pyvi**: Thư viện tiếng Việt hóa văn bản

In [ ]:
import re
import demoji
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, GRU, Input, GlobalMaxPooling1D, LayerNormalization, Conv1D, MaxPooling1D, ELU
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pyvi import ViTokenizer, ViUtils

demoji.download_codes()

# Phần 2: Xử lý dữ liệu

Nhập liệu

In [ ]:
data = pd.read_csv('data_modified.csv')
missing_values = data.isnull().sum()


print(f"Số lượng hàng: {data.shape[0]}")
print(f"Số lượng cột: {data.shape[1]}")
print(f"Tên cột: {list(data.columns)}")
print(f"Thông tin cột bị thiếu: \n{missing_values}")
data.info()

sentiment_data = data
sentiment_data.head(5)

In [ ]:
pos_data = sentiment_data[sentiment_data['label'] == 'POS']
pos_data

In [ ]:
neu_data = sentiment_data[sentiment_data['label'] == 'NEU']
neu_data

In [ ]:
neg_data = sentiment_data[sentiment_data['label'] == 'NEG']
neg_data

Tiền xử lý 

In [ ]:
def remove_emo(text):
    emo = demoji.replace(text, '')
    return emo

def remove_urls(text):
    text = re.sub(r'http\S+', '', text)
    return text

def remove_mentions_and_emails(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    return text

In [ ]:
input_data = sentiment_data['comment'].values
input_label = sentiment_data['label'].values

label_dict = {'NEG':0,'NEU':1,'POS':2}

input_pre = []
label_with_accent = []
for idx, dt in enumerate(input_data):
    #Chuyển đổi thành list
    input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(dt))
    # Chuyển danh sách từ 1 chuỗi
    input_text_pre = " ".join(input_text_pre)

    # Tiền xử lý dữ liệu
    input_text_pre = remove_emo(input_text_pre)  
    input_text_pre = remove_urls(input_text_pre)  
    input_text_pre = remove_mentions_and_emails(input_text_pre)

    # Tách dấu
    input_text_pre_no_accent = str(ViUtils.remove_accents(input_text_pre).decode("utf-8"))

    # Tách từ
    input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
    input_text_pre_no_accent = ViTokenizer.tokenize(input_text_pre_no_accent)

    input_pre.append(input_text_pre_accent)
    input_pre.append(input_text_pre_no_accent)
    label_with_accent.append(input_label[idx])
    label_with_accent.append(input_label[idx])
input_pre

Biểu đồ histogram dữ liệu từ 0 đến 31k

In [ ]:
bin_count = 10  # Số lượng bin
chunk_size = 1000   # Kích thước của mỗi phân đoạn
total_chunks = 10000 // chunk_size      # Tổng số phân đoạn

# Tạo 1 figure mới
fig, axes = plt.subplots(nrows=(total_chunks // 6) + (1 if total_chunks % 6 != 0 else 0), ncols=6, figsize=(20, (total_chunks // 6) * 5))

for i, ax in zip(range(0, 31000, chunk_size), axes.flatten()):
    seq_len = [len(sentence.split()) for sentence in input_pre[i:i+chunk_size]]     # Tính độ dài của từng câu
    pd.Series(seq_len).hist(bins=bin_count, ax=ax)      # Vẽ histogram
    ax.set_title(f'{i} đến {i+chunk_size}')     # Tiêu đề

# Xóa các trục con thừa
for j in range(total_chunks, len(axes.flatten())):
    fig.delaxes(axes.flatten()[j])

plt.tight_layout()
plt.show()

In [ ]:
# chuyển đổi các nhãn từ dạng chuỗi sang dạng số
label_idx = [label_dict[i] for i in label_with_accent]
# mã hóa one-hot
label_tf = tf.keras.utils.to_categorical(label_idx, num_classes=3)
label_tf = label_tf.astype('float32')

# xử lý văn bản thành các chuỗi số và đệm
tokenizer_data = Tokenizer(oov_token='<OOV>', filters='', split=' ')    # tạo từu khóa OOV, không lọc ký tự cách từ = dấu cách
tokenizer_data.fit_on_texts(input_pre)  # học từ điển từ danh sách đã tiền xử lý
 
tokenizer_data_text = tokenizer_data.texts_to_sequences(input_pre)      # vb => chuỗi số
vec_data = pad_sequences(tokenizer_data_text, padding='post', maxlen=512) # đệm chuỗi số sang dạng cố định 512

#lưu tokenizer
pickle.dump(tokenizer_data, open("tokenizer_data.pkl","wb"))

print(f"Kích thước dữ liệu đầu vào: {vec_data.shape}")
data_vocab_size = len(tokenizer_data.word_index)+1  # kích thước từ điển + từ khóa OOV
print(f"Kích thước của từ điển: {data_vocab_size}")

X_train, X_val, y_train, y_val = train_test_split(vec_data, label_tf, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print(f"Số mẫu trong tập huấn luyện: {len(X_train)}")
print(f"Số mẫu trong tập validation: {len(X_val)}")
print(f"Số mẫu trong tập kiểm tra: {len(X_test)}")


# Phần 3: Train model

In [ ]:
def generate_model():
    dropout_threshold = 0.5     # Ngưỡng dropout
    input_dim = data_vocab_size     # Kích thước từ vựng
    output_dim = 32    # Kích thước của vector embedding
    input_length = 512  # Độ dài tối đa
    initializer = tf.keras.initializers.GlorotNormal()

    input_layer = Input(shape=(input_length,))
    # chuyển đổi từ khóa vào vector
    feature = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length, embeddings_initializer=initializer)(input_layer)

    # CNN để trích xuất đặc trưng
    cnn_feature = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(feature)
    cnn_feature = MaxPooling1D()(cnn_feature)
    cnn_feature = Dropout(dropout_threshold)(cnn_feature)
    cnn_feature = Conv1D(filters= 32, kernel_size=3, padding='same', activation='relu')(cnn_feature)
    cnn_feature = MaxPooling1D()(cnn_feature)
    cnn_feature = LayerNormalization()(cnn_feature)
    cnn_feature = Dropout(dropout_threshold)(cnn_feature)

    # Mạng Bi-directional LSTM để xử lý tuần tự dữ liệu
    bi_lstm_feature = Bidirectional(LSTM(units=32, dropout=dropout_threshold, return_sequences=True, kernel_initializer=initializer), merge_mode='concat')(feature)
    bi_lstm_feature = MaxPooling1D()(bi_lstm_feature)
    bi_lstm_feature = Bidirectional(GRU(units=32, dropout=dropout_threshold, return_sequences=True, kernel_initializer=initializer), merge_mode='concat')(bi_lstm_feature)
    bi_lstm_feature = MaxPooling1D()(bi_lstm_feature)
    
    bi_lstm_feature = LayerNormalization()(bi_lstm_feature)

    # Kết hợp đặc trưng từ CNN và Bi-LSTM
    combine_feature = tf.keras.layers.Concatenate()([cnn_feature, bi_lstm_feature])
    combine_feature = GlobalMaxPooling1D()(combine_feature)
    combine_feature = LayerNormalization()(combine_feature)

    # Các lớp Dense (MLP) để phân loại
    classifier = Dense(90, activation='relu')(combine_feature)
    classifier = Dropout(0.3)(classifier)
    classifier = Dense(70, activation='relu')(classifier)
    classifier = Dropout(0.3)(classifier)
    classifier = Dense(50, activation='relu')(classifier)
    classifier = Dropout(0.3)(classifier)
    classifier = Dense(30, activation='relu')(classifier)
    classifier = Dropout(0.3)(classifier)
    classifier = Dense(10, activation='relu')(classifier)
    classifier = Dropout(0.3)(classifier)
    classifier = Dense(3, activation='softmax')(classifier)

    # Tạo mô hình Keras
    model = Model(inputs=input_layer, outputs=classifier)

    return model

model = generate_model()
adam = Adam(learning_rate=0.001)   
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Huấn luyện

In [ ]:
# Định nghĩa callback để lưu mô hình
callback_model = tf.keras.callbacks.ModelCheckpoint(
    'model_cnn_bilstm.keras',
    monitor='val_loss',
    save_best_only=True  # Chỉ lưu mô hình tốt nhất
)

# Huấn luyện mô hình
history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=128,
    callbacks=[callback_model]
)

In [ ]:
model.load_weights("model_cnn_bilstm.keras")
model.evaluate(X_test,y_test)

In [ ]:
# Trực quan hóa hàm mất mát
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Hàm mất mát của tập train và tập validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Trực quan hóa độ chính xác
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Độ chính xác của tập train và tập validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Phần 4: Chạy Model

In [ ]:
# Hàm tiền xử lý đầu vào văn bản th
def preprocess_raw_input(raw_input, tokenizer):
    input_text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(raw_input))
    input_text_pre = " ".join(input_text_pre)
    input_text_pre_accent = ViTokenizer.tokenize(input_text_pre)
    print(f"Comment: {input_text_pre_accent}")
    tokenizer_data_text = tokenizer.texts_to_sequences([input_text_pre_accent])
    vec_data = pad_sequences(tokenizer_data_text, padding='post', maxlen=512)
    return vec_data

# Hàm này nhận đầu vào đã tiền xử lý và mô hình để thực hiện suy diễn và trả về kết quả dự đoán
def inference_model(input_feature, model):
    output = model(input_feature).numpy()[0]    # Dự đoán bằng mô hình
    result = output.argmax()        # Lấy nhãn có xác suất cao nhất
    conf = float(output.max())  
    label_dict = {'NEG': 0, 'NEU': 1, 'POS': 2}         # Từ điển các nhãn
    label = list(label_dict.keys())
    return label[int(result)], conf

# Hàm  kết hợp tiền xử lý và suy diễn 
def prediction(raw_input, tokenizer, model):
    input_model = preprocess_raw_input(raw_input, tokenizer)        # Tiền xử lý đầu vào
    # Dự đoán kết quả
    result, conf = inference_model(input_model, model)
    return result, conf

# Tạo và tải mô hình
my_model = generate_model()
my_model = load_model('model_cnn_bilstm.keras')

# Tải tokenizer từ file
with open(r"tokenizer_data.pkl", "rb") as input_file:
    my_tokenizer = pickle.load(input_file)


In [ ]:
# Hàm để phân loại các bình luận từ file CSV
def classify_comments_from_csv(file_path, tokenizer, model):
    df = pd.read_csv(file_path)
    comments = df['comment']  # Giả sử cột chứa bình luận có tên là 'comment'
    
    results = []
    for comment in comments:
        result, conf = prediction(comment, tokenizer, model)
        results.append((comment, result, conf))
    
    result_df = pd.DataFrame(results, columns=['comment', 'label', 'confidence'])
    return result_df

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd

history_list = []

def on_predict():
    user_input = input_text.get("1.0", tk.END).strip()
    if user_input:
        result, confidence = prediction(user_input, my_tokenizer, my_model)
        result_label.config(text=f"Kết quả bình luận: {result}")
        save_history(user_input, result)

def save_history(comment, result):
    history_list.append((comment, result))

def delete_history():
    global history_list
    history_list = []
    show_history()

def export_history():
    if not history_list:
        return
    file_path = filedialog.asksaveasfilename(defaultextension='', filetypes=[('CSV files', '*.csv'), ('Text files', '*.txt'), ('Excel files', '*.xlsx')])
    if file_path:
        df = pd.DataFrame(history_list, columns=['Bình luận', 'Kết quả'])
        if file_path.endswith('.csv'):
            df.to_csv(file_path, index=False)
        elif file_path.endswith('.txt'):
            df.to_csv(file_path, index=False, sep=',')
        elif file_path.endswith('.xlsx'):
            df.to_excel(file_path, index=False)
        else:
            tk.messagebox.showerror('Lỗi', 'Định dạng tệp không được hỗ trợ')
            return
        tk.messagebox.showinfo('Lưu thành công', f'Dữ liệu đã được lưu vào {file_path}')

def show_history():
    history_window = tk.Toplevel(root)
    history_window.title("Lịch sử")
    history_window.geometry("280x350")
    history_window.resizable(width=False, height=False)
    
    if not history_list:
        ttk.Label(history_window, text="Chưa có dữ liệu lịch sử.").pack(padx=10, pady=10)
    else:
        frame_P = ttk.Frame(history_window, borderwidth=3, relief="solid", width=280, height=280)
        frame_P.pack(side='top', pady=5)
        frame_P.pack_propagate(False)
        frame_tree = ttk.Frame(frame_P, borderwidth=3, relief="flat", width=280, height=256)
        frame_tree.pack(side='top')
        frame_tree.pack_propagate(False)
        tree_frame = ttk.Frame(frame_tree, borderwidth=0, relief="flat", width=250, height=256)
        tree_frame.pack(side="left", expand=True)
        tree_frame.pack_propagate(False)
        text_widget = tk.Text(tree_frame, wrap='word', width=42, height=17)
        text_widget.pack(side='top',expand=True)
        scrollbar_y = tk.Scrollbar(frame_tree, orient="vertical")
        scrollbar_y.pack(side="right", fill="y")
        scrollbar_x = tk.Scrollbar(frame_P, orient="horizontal")
        scrollbar_x.pack(side="top", fill="x")
        text_widget.configure(yscrollcommand=scrollbar_y.set)
        text_widget.configure(xscrollcommand=scrollbar_x.set)
        for i, (comment, result) in enumerate(history_list, start=1):
            text_widget.insert('end', f"Bình luận {i}: {comment}\nKết quả: {result}\n\n")

        text_widget.config(state='disabled')

    frame_history = ttk.Frame(history_window, borderwidth=0, relief="solid", width=380, height=50)
    frame_history.pack(side='top')
    frame_history.pack_propagate(False)
    ttk.Button(frame_history, text="Xóa lịch sử", command=delete_history).pack(side='left', ipadx=10, ipady=5, fill='both', expand=True)
    ttk.Button(frame_history, text="Lưu lịch sử", command=export_history).pack(side='left', ipadx=10, ipady=5, fill='both', expand=True)

# Hàm để tải file CSV
def upload_file():
    file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if file_path:
        global result_df
        try:
            result_df = classify_comments_from_csv(file_path, my_tokenizer, my_model)
            label_configure.config(text="File đã được tải lên")
        except Exception as e:
            tk.messagebox.showerror("Lỗi 404", f"Không thể xử lý tệp: {e}")
            label_configure.config(text="Vui lòng tải lại.")
    else:
        label_configure.config(text="Không có tệp được chọn.")

# Hàm để xuất file CSV
def export_file():
    if 'result_df' in globals():
        file_path = filedialog.asksaveasfilename(defaultextension=".csv", 
                                                 filetypes=[("CSV Files", "*.csv"), 
                                                            ("Text Files", "*.txt"), 
                                                            ("Excel Files", "*.xlsx")])
        if file_path:
            try:
                if file_path.endswith('.csv'):
                    result_df.to_csv(file_path, index=False)
                elif file_path.endswith('.txt'):
                    result_df.to_csv(file_path, sep=',', index=False)
                elif file_path.endswith('.xlsx'):
                    result_df.to_excel(file_path, index=False)
                label_configure.config(text="Lưu tệp thành công")
            except Exception as e:
                tk.messagebox.showerror("Lỗi 400", f"Không thể xuất tệp: {e}")
                label_configure.config(text="Vui lòng thử lại")
    else:
        tk.messagebox.showwarning("Warning", "Không có kết quả để xuất. Vui lòng tải lên và xử lý tệp trước.")
        label_configure.config(text="Vui lòng tải lên và xử lý tệp trước.")

# Hàm xóa nội dung văn bản
def clear_text():
    input_text.delete("1.0", tk.END)
    result_label.config(text="")


# Tạo cửa sổ chính
root = tk.Tk()
root.title("Phân tích cảm xúc bằng văn bản")
root.iconbitmap("img/Logo.ico")
root.geometry("600x450")
root.configure(bg="#f0f0f0")
root.resizable(width=False, height=False)


#-----------------------------------------------------------------------------------------------------------------------------------
frame_P1 = ttk.Frame(root, borderwidth=0, relief="flat", width=380, height=50)
frame_P1.pack(side='top')
frame_P1.pack_propagate(False)

# Tiêu đề
title_label = ttk.Label(frame_P1, text="Phân tích cảm xúc bằng văn bản", font=("Helvetica", 16))
title_label.pack(side='left', padx=5, pady=10)

gifImage = "img/decorate_duck.gif"
openImage = Image.open(gifImage)

new_width = 50
new_height = 55
imageObject = []
for frame_num in range(openImage.n_frames):
    openImage.seek(frame_num)
    resized_frame = openImage.resize((new_width, new_height), Image.Resampling.LANCZOS)
    imageObject.append(ImageTk.PhotoImage(resized_frame))
count = 0

def animation(count):
    newImage = imageObject[count]
    gif_Label.configure(image=newImage)
    count += 1
    if count == openImage.n_frames:
        count = 0
    frame_P1.after(50, lambda: animation(count))

gif_Label = ttk.Label(frame_P1, image="")
gif_Label.pack(side='left', padx=1, pady=1)
frame_P1.after(50, lambda: animation(count))

#-----------------------------------------------------------------------------------------------------------------------------
frame_P2 = ttk.Frame(root, borderwidth=0, relief="flat", width=600, height=230)
frame_P2.pack(side='top')
frame_P2.pack_propagate(False)

# Hướng dẫn sử dụng
help_frame = ttk.Frame(frame_P2, padding="10", borderwidth=2, relief="solid")
help_frame.pack(padx=10, pady=5, fill="x")

help_label = ttk.Label(help_frame, text="Hướng dẫn:", font=("Helvetica", 14))
help_label.pack(pady=5, anchor='w')

normal_font = ('Helvetica', 10)
help_gt = (
    "- Hãy nhập bất kỳ bình luận nào liên quan đến mua sắm sản phẩm trên các sàn thương mại điện tử.\n"
    "- Hệ thống sẽ phân tích và trả về kết quả theo 3 mức độ cảm xúc: NEG, NEU và POS.\n"
    "  Chi tiết như sau:\n"
    "    ~ NEG: Bình luận mang tính tiêu cực\n"
    "    ~ NEU: Bình luận mang tính trung lập\n"
    "    ~ POS: Bình luận mang tính tích cực\n"
    "* Mức độ chính xác xấp xỉ khoảng 60 - 80%"
    "- Hãy thử ngay và khám phá xem bình luận của bạn thuộc mức độ nào!"
)

help_gioithieu_content = ttk.Label(help_frame, text=help_gt, font=normal_font, wraplength=550)
help_gioithieu_content.pack(pady=5, anchor='w')

#-------------------------------------------------------------------------------------------------------------------------
frame_P3 = ttk.Frame(root, borderwidth=0, relief="flat", width=600, height=80)
frame_P3.pack(side='top')
frame_P3.pack_propagate(False)

# Nút để bắt đầu phân tích và xóa
text_frame = ttk.Frame(frame_P3,relief="flat", width=450, height=70)
text_frame.pack(side="left", padx=10)
text_frame.pack_propagate(False)

label = ttk.Label(text_frame, text="Nhập văn bản:", font=("Helvetica", 10))
label.pack(side='top', anchor='w', padx=5, pady=5)

# Text box để nhập văn bản
input_text = tk.Text(text_frame, height=1, width=45, font=("Helvetica", 12))
input_text.pack(side="top",padx=10, pady=5)

# Nút để bắt đầu phân tích và xóa
button_frame = ttk.Frame(frame_P3,relief="solid", width=120, height=70)
button_frame.pack(side="right", padx=10)
button_frame.pack_propagate(False)

analyze_button = ttk.Button(button_frame, text="Phân tích cảm xúc", command=on_predict)
analyze_button.pack(side='top', ipadx=10, ipady=5, fill='both', expand=True)

clear_button = ttk.Button(button_frame, text="Xóa", command=clear_text)
clear_button.pack(side='top', ipadx=10, ipady=5, fill='both', expand=True)

#-----------------------------------------------------------------------------------------------------------------------------
frame_P4 = ttk.Frame(root, borderwidth=0, relief="flat", width=580, height=40)
frame_P4.pack(side='top')
frame_P4.pack_propagate(False)

result_label = ttk.Label(frame_P4, text="Kết quả bình luận:", font=("Helvetica", 10), wraplength=550, background="#f0f0f0")
result_label.pack(side='left',padx=10,pady=10)

history_button = ttk.Button(frame_P4, text="Lịch sử", command=show_history)
history_button.pack(side='right',padx=5, ipadx=15, ipady=5)

#--------------------------------------------------------------------------------------------------------------------
frame_P5 = ttk.Frame(root, borderwidth=0, relief="solid", width=580, height=40)
frame_P5.pack(side='top',pady=5)
frame_P5.pack_propagate(False)

frame_button_upload = ttk.Frame(frame_P5, relief="flat", width=320, height=40)
frame_button_upload.pack(side="left", padx=5)
frame_button_upload.pack_propagate(False)

# Tạo nút để tải file CSV
upload_button = ttk.Button(frame_button_upload, text="Upload CSV", command=upload_file)
upload_button.pack(side="left",padx=5,pady=5,fill='both', expand=True)

# Tạo nút để xuất file CSV
export_button = ttk.Button(frame_button_upload, text="Export CSV", command=export_file)
export_button.pack(side="left",padx=5,pady=5, fill='both', expand=True)

label_configure = ttk.Label(frame_P5, text="Tải file chứa bình luận", wraplength=400)
label_configure.pack(side='right',padx=5,pady=10, fill='both', expand=True)

# Chạy ứng dụng
root.mainloop()